In [2]:
# Cell 1：快取直方圖（Precompute histograms）
from train_pf_batchVer import collect_histograms_from_folder_ply
from tqdm.notebook import tqdm

# 你要測的 bins 列表
bins_list = [32, 64, 128]

# 快取直方圖的 dict
hist_cache = {}
for num_bins in tqdm(bins_list, desc="Precompute bins"):
    # collect_histograms 自帶每個 ply 的進度條
    Hs_train = collect_histograms_from_folder_ply("../dataset/test", num_bins)
    Hs_val   = collect_histograms_from_folder_ply("../dataset/val",   num_bins)
    hist_cache[num_bins] = (Hs_train, Hs_val)

# 現在 hist_cache 裡面有每個 num_bins 對應的 (Hs_train, Hs_val)


Precompute bins:   0%|          | 0/3 [00:00<?, ?it/s]

Overall Progress: 100%|██████████| 2400/2400 [43:52<00:00,  1.10s/it]
處理 model_vox11_00000600.ply: 
Overall Progress: 100%|██████████| 302/302 [11:28<00:00,  2.28s/it]
處理 soldier_vox10_0833.ply: 
Overall Progress: 100%|██████████| 2400/2400 [41:56<00:00,  1.05s/it]
處理 model_vox11_00000600.ply: 
Overall Progress: 100%|██████████| 302/302 [11:07<00:00,  2.21s/it]
處理 soldier_vox10_0833.ply: 
Overall Progress: 100%|██████████| 2400/2400 [42:00<00:00,  1.05s/it]
處理 model_vox11_00000600.ply: 
Overall Progress: 100%|██████████| 302/302 [11:11<00:00,  2.22s/it]
處理 soldier_vox10_0833.ply: 


In [3]:
# Cell 2：載入 train_pf，迴圈跑不同的 (K, bin) 組合
import os
from train_pf_batchVer import train_pf
from tqdm.notebook import tqdm

# 參數表：bin → K 值列表
param_grid = {
    64:  [128],
    128: [1, 2, 4, 8],
    32:  [2, 4, 8, 16, 32, 64, 128],
    16:  [1, 2, 4, 8, 16, 32, 64, 128],
    8:   [1, 2, 4, 8, 16, 32, 64, 128],
    4:   [1, 2, 4, 8, 16, 32, 64, 128],
    2:   [1, 2, 4, 8, 16, 32, 64, 128],
    1:   [1, 2, 4, 8, 16, 32, 64, 128],
}

output_dir    = "pth"
os.makedirs(output_dir, exist_ok=True)
save_template = "mppq_K{K}_bins{bin}.pth"

# 外層：不同 bin，顯示進度
for num_bins in tqdm(param_grid.keys(), desc="Bins to train"):
    Hs_train, Hs_val = hist_cache[num_bins]
    # 內層：不同 K，顯示進度但跑完自動收起
    for K in tqdm(param_grid[num_bins], desc=f"K values (bin={num_bins})", leave=False):
        save_path = os.path.join(output_dir, save_template.format(K=K, bin=num_bins))
        train_pf(
            K=K,
            num_bins=num_bins,
            epochs=200,
            lr=1e-3,
            clip_norm=1.0,
            use_huber=False,
            max_moment=None,
            batch_size=32,
            save_path=save_path,
            Hs_train=Hs_train,
            Hs_val=Hs_val
        )


Bins to train:   0%|          | 0/8 [00:00<?, ?it/s]

K values (bin=64):   0%|          | 0/1 [00:00<?, ?it/s]

TypeError: train_pf() got an unexpected keyword argument 'Hs_train'